In [ ]:
# libraries (imported in order of use in the notebook)
import mikeio
# simple simulation functionality: RunTimeEvaluation, Launcher
from mike_autocal.mikesimulation import RunTimeEvaluation, Launcher
# error tracking during runtime
from mike_autocal.dataio import ObservationData, SimObsPair, SimulationData
from mike_autocal.objective_fun import RMSEInnerMetric, AMEANOuterMetric
# automatic calibration 
import optuna
from mike_autocal.measurement_fun import ManningFile
from mike_autocal.autocal import AutoCal



# directories
from pathlib import Path
ROOT_DIR = Path().resolve()

# 1. Load and investigate setup

This demo is based on the [WaterBench Southern North Sea](https://github.com/DHI/WaterBench-MIKE21HD-SouthernNorthSea) model and observations.

To briefly inspect the setup we leverage [MIKEIO](https://github.com/DHI/mikeio) and its pfs reading capabilities

In [ ]:
# specify simulation setup file 
simfile = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/simulation/sns_base.m21fm"
# read with mikeio and display some information
pfs = mikeio.read_pfs(simfile)
pfs.FemEngineHD.TIME

# 2. Run simulation with simple logging

* simplest version with basic logging of runtime information (from stdout, interpolation in tensorboard if output frequency is lower than logging frequency)
* without utilizing observations

* introducing Tensorboard and `RunTimeEvaluation` (RTE) from `mike_autocal`. 
    * Generates logs readable and renderable with TensorBoard
    * Logs can be found in `./logs` and accessed via Tensorboard in real time.
    * frequency is the only argument that can be used in the RTE when no observation (SimObsPairs) are specified.
    * Logs will solely contain runtime information  

* introducing simulation `Launcher` from `mike_autocal` 


In [ ]:
rte = RunTimeEvaluation(frequency=10) # frequency of logging will be every 10 timesteps by default  


launcher = Launcher( 
    simfile = simfile,      # path to simulation setup file
    use_gpu=True,           # use GPU if available
    runtimeevaluation=rte,  # Use an empty RunTimeEvaluation object
    n_cores = 1)
    
launcher.execute_simulation();

# 2. Extend logging with error tracking from in-situ observations

* we need to link observations to simulation output $\rightarrow$ introducing `SimObsPairs` 


In [ ]:
base_sim_path = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/simulation/sns_base.m21fm - Result Files"
base_obs_path = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/observations"

simobs = [
    SimObsPair(
        name="F3platform",
        pair_type="point",
        sim=SimulationData(
            file_path = base_sim_path / "waterlevels.dfs0",
            item=7,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "F3platform_wl.dfs0",
            item=0,  
        ),
    ),
    SimObsPair(
        name="Helgoland",
        pair_type="point",
        sim=SimulationData(
            file_path = base_sim_path / "waterlevels.dfs0",
            item=5,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "Helgoland_wl.dfs0",
            item=0,  
        ),
    ), # add as many as you want
    ]

using rte and the launcher once again, slightly extended.
* extended RTE allows for live tracking of specified error metrics
* introducing `InnerMetric` as well as `OuterMetric`
    * `InnerMetric` is used to calculate the error between the simulation and the observation for each station
    * `OuterMetric` is the aggregation of the inner metrics

In [ ]:
rte = RunTimeEvaluation(simobs=simobs, 
                        inner_metric=[RMSEInnerMetric()], # can be multiple metrics as well (e.g. CC) 
                        outer_metric=[AMEANOuterMetric()], # defines how to aggregate data from multiple stations
                        frequency=10)


launcher = Launcher(
    simfile = simfile,      # path to simulation setup file
    use_gpu=True,           # use GPU if available
    runtimeevaluation=rte,  # Use new RunTimeEvaluation object
    n_cores = 1)
    
launcher.execute_simulation();

# 3. Automatic calibration guided by error metrics


* What do we want to optimize (e.g. Manning's n map)?
* This is defined in `measurement_functions`


In [ ]:
# load and inspect Manning's n map with existing zonation
manningfile = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/simulation/conditions/ManningM.dfsu"
mikeio.read(manningfile)[0].plot();

* introducing Optuna for parameter estimation (automatic calibration)


In [ ]:
inner_metric = [RMSEInnerMetric()] # can be multiple metrics as well (e.g. CC) 
outer_metric = [AMEANOuterMetric()] # defines how to aggregate data from multiple stations

# Optimization
sampler = optuna.samplers.GPSampler(seed=0) # which optimization algorithm to use (defaults to TPE)
evaluation_time = slice(50, None)           # at which time steps to evaluate the objective function (it can be a good idea to leave the swing-in period out)

direction = ["minimize"]                    # direction of optimization (see inner metric), can be multiobjective 
n_trials = 5                               # number of simulation runs in optimization
study_name = "testing_calibration"          # recognizable name


measurement_functions = [
    ManningFile(
        filename= manningfile,
        item_name="manning",
        low=0.001,
        high=81.101,
        step=0.01,
    )]

* introducing `AutoCal` for automatic calibration with goal to 
    * optimize Error on specified metric(s)
    * by modification of measurement functions (parameters) 


In [ ]:
calibration = AutoCal(
    launcher=launcher,          # already contains simfile
    # add observations and metrics
    simobs=simobs,
    inner_metric=inner_metric,  
    outer_metric=outer_metric,
    # optimization settings
    study_name=study_name,
    n_trials=n_trials,
    direction=direction,
    sampler=sampler, 
    measurement_functions=measurement_functions,
    evaluation_time=evaluation_time,
    verbose=False,
    load_if_exists=True,
)
calibration.run()

* For more optimization details during runtime visit optuna dashboard on calibration log: `optuna-dashboard optuna_journal.log`

# 4. Evaluate simulations

* use optuna dataframe, 
* illustrate optimization based on simulation waterlevel (color sequentially by trial number)

In [ ]:
file_path = "optuna_journal.log"
storage = optuna.storages.JournalStorage(
    optuna.storages.journal.JournalFileBackend(file_path)
)

study_name = "testing_calibration"  # Use the actual study name used when creating the study
study = optuna.load_study(study_name=study_name, storage=storage)

# 3. Convert the trials to a DataFrame
df = study.trials_dataframe()
df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
# use F3 platform observation and all simulations
base_obs_path = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/observations"
base_sim_path = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/simulations"

fig, ax = plt.subplots(1,1,figsize=(10,5))

# Get a sequential colormap (cool)
cmap = cm.cool
trial_nums = df.number.values
norm = plt.Normalize(min(trial_nums), max(trial_nums))

# Clear any existing plots on the axis
ax.clear()

for i, tnum in enumerate(trial_nums):
    sim = mikeio.read(str(simfile).split(".")[0] + "_" + study_name + f"_trial_{tnum}.m21fm - Result Files/waterlevels.dfs0", items=["F3platform: Surface elevation"]).to_dataframe()
    # Use the colormap to assign colors based on trial number
    color = cmap(norm(tnum))
    
    # Plot without labels for trials
    ax.plot(sim.index, sim.values, color=color)

obs = mikeio.read(base_obs_path / "F3platform_wl.dfs0").to_dataframe()
# Plot observed data with explicit label
obs_line = ax.plot(obs.loc[sim.index].index, obs.loc[sim.index].values, c="black", linewidth=2)
ax.legend([obs_line[0]], ["Observed"], loc='upper right')

# Add a colorbar instead of a legend for trials
cax = fig.add_axes([0.15, 0.05, 0.7, 0.03])  # [left, bottom, width, height]
cb = ColorbarBase(cax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Trial Number')

# Adjust figure to make room for colorbar
plt.subplots_adjust(bottom=0.15)